<a href="https://colab.research.google.com/github/elbernaderen/final-project-cs50/blob/master/final_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Instalación de python, librerias con mapas y flask en el servidor

In [1]:
# Important library for many geopython libraries
!apt install gdal-bin python-gdal python3-gdal 



Reading package lists... Done
Building dependency tree       
Reading state information... Done
Package python-gdal is not available, but is referred to by another package.
This may mean that the package is missing, has been obsoleted, or
is only available from another source
However the following packages replace it:
  gdal-bin

E: Package 'python-gdal' has no installation candidate


In [2]:
# Install rtree - Geopandas requirment
!apt install python3-rtree 


Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-510
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  javascript-common libjs-jquery libjs-underscore libspatialindex-c6
  libspatialindex-dev libspatialindex6
Suggested packages:
  apache2 | lighttpd | httpd
The following NEW packages will be installed:
  javascript-common libjs-jquery libjs-underscore libspatialindex-c6
  libspatialindex-dev libspatialindex6 python3-rtree
0 upgraded, 7 newly installed, 0 to remove and 21 not upgraded.
Need to get 829 kB of archives.
After this operation, 3,054 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal/main amd64 javascript-common all 11 [6,066 B]
Get:2 http://archive.ubuntu.com/ubuntu focal/main amd64 libjs-jquery all 3.3.1~dfsg-3 [329 kB]
Get:3 http://archive.ubuntu.com/ubuntu 

In [3]:
# Install descartes - Geopandas requirment
!pip install descartes 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
# Install Folium for Geographic data visualization
!pip install folium


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
# Install plotlyExpress
!pip install plotly_express


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
# Install flask-ngrok to run flask on colab
!pip install flask-ngrok
# 2023
!pip install pyngrok


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 761.3/761.3 KB 11.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-5.2.1-py3-none-any.whl size=19792 sha256=3cd78a8ecbb30971f392795e1437a1714a17ec48fe4427766468eff95e5a2e46
  Stored in directory: /root/.cache/pip/wheels/5d/f2/70/526da675d32f17577ec47ac4c663084efe39d47c826b6c3bb1
Successfully built pyngrok


In [7]:
# Installjavascript on colab
!npm install -g --unsafe-perm ijavascript zeromq
!ijsinstall --install=global
!.npm install -g npm

npm WARN deprecated uuid@3.4.0: Please upgrade  to version 7 or higher.  Older versions may use Math.random() in certain circumstances, which is known to be problematic.  See https://v8.dev/blog/math-random for details.
/tools/node/bin/ijs -> /tools/node/lib/node_modules/ijavascript/bin/ijavascript.js
/tools/node/bin/ijsconsole -> /tools/node/lib/node_modules/ijavascript/bin/ijsconsole.js
/tools/node/bin/ijsinstall -> /tools/node/lib/node_modules/ijavascript/bin/ijsinstall.js
/tools/node/bin/ijskernel -> /tools/node/lib/node_modules/ijavascript/lib/kernel.js
/tools/node/bin/ijsnotebook -> /tools/node/lib/node_modules/ijavascript/bin/ijsnotebook.js

> zeromq@5.3.1 install /tools/node/lib/node_modules/ijavascript/node_modules/zeromq
> node-gyp-build || npm run build:libzmq

make: Entering directory '/tools/node/lib/node_modules/ijavascript/node_modules/zeromq/build'
  ACTION binding_gyp_libzmq_target_prepare_build zmq/BUILD_SUCCESS
Building libzmq for linux
build_libzmq.sh: Building vers

In [8]:
! pip install Flask-Session
! pip install Flask

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [9]:
# 2023
!git clone https://github.com/geopandas/geopandas.git
!cd geopandas
!pip install geopandas

Cloning into 'geopandas'...
remote: Enumerating objects: 13113, done.
remote: Counting objects: 100% (362/362), done.
remote: Compressing objects: 100% (245/245), done.
remote: Total 13113 (delta 205), reused 206 (delta 116), pack-reused 12751
Receiving objects: 100% (13113/13113), 56.70 MiB | 16.71 MiB/s, done.
Resolving deltas: 100% (8833/8833), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 28.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 KB 10.6 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 7.1.2
    Uninstalling click-7.1.2:
      Successfully uninstalled click-7.1.2
ERROR: pip's dependency resolver does not currently take into account all the

2023- Ngrok requirement: se agrega el token en un archivo .yml

In [10]:
import yaml
from pyngrok import ngrok

config = yaml.load(open("/content/drive/MyDrive/final/ignore/config.yml"), Loader = yaml.FullLoader)
!ngrok authtoken config["key"]

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


# Base de datos

In [11]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS claims(id_claim INTEGER,
                      name TEXT,
                      address TEXT, 
                      latitude REAL, 
                      longitude REAL, 
                      date TEXT,
                      reason TEXT,
                      PRIMARY KEY(id_claim));''')

conn.commit()

print("Table created successfully");

conn.close()

Opened database successfully
Table created successfully


In [12]:
import sqlite3

conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db')
print("Opened database successfully");

conn.execute('''
CREATE TABLE IF NOT EXISTS users (id INTEGER,
                              username TEXT,
                              hash TEXT, 
                              PRIMARY KEY(id));''')
conn.commit()

print("Table created successfully");

conn.close()

Opened database successfully
Table created successfully


compruebo que esté instalado geopandas (tarda en encontrar el modulo)

In [13]:
import sys
sys.path.append('/content/drive/MyDrive/final')
'geopandas' in sys.modules

False

# aplicación

https://stackoverflow.com/questions/52638926/geo-coding-for-intersection-street-using-python from here i have taken the idea for searching latitude and longitude from googlemaps

In [ ]:
import os
import sys
sys.path.append('/content/drive/MyDrive/final')
import urllib.parse
import string
from flask_ngrok import run_with_ngrok
from flask_session import Session
from datetime import datetime
from tempfile import mkdtemp
from flask import Flask, flash, redirect, render_template, request, session, make_response,g
from flask import send_file

from geopandas.tools import geocode
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster
import requests
import sqlite3
import re
from werkzeug.exceptions import default_exceptions, HTTPException, InternalServerError
from werkzeug.security import check_password_hash, generate_password_hash
from helpers import login_required
from pytz import timezone
import random
import json
import csv



app = Flask(__name__,template_folder='/content/drive/MyDrive/final/template')
run_with_ngrok(app)
app.config["TEMPLATES_AUTO_RELOAD"] = True



# Ensure responses aren't cached
@app.after_request
def after_request(response):
    response.headers["Cache-Control"] = "no-cache, no-store, must-revalidate"
    response.headers["Expires"] = 0
    response.headers["Pragma"] = "no-cache"
    return response



app.config["SESSION_FILE_DIR"] = mkdtemp()
app.config["SESSION_PERMANENT"] = False
app.config["SESSION_TYPE"] = "filesystem"
Session(app)



lat = -31.4186703
long = -64.1903681


@app.route("/", methods=["GET", "POST"])
def index():
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        m_1 = folium.Map(location = (lat,long), zoom_start = 12)
        m_1.save('/content/drive/MyDrive/final/template/map.html')
        if request.method == "POST":
          name = request.form.get("name")
          time = datetime.now(timezone("America/Argentina/Cordoba"))
          direction = request.form.get("direction")
          reason =  request.form.get("reason")
          gps1 = loc(direction)
          gps1[0] = float(gps1[0])
          gps1[1] = float(gps1[1])
          conn.execute("INSERT INTO claims (name, address, latitude, longitude, date, reason) VALUES(?, ?, ?, ?, ?, ?)", [name, direction, gps1[0], gps1[1], time, reason])
          conn.row_factory = sqlite3.Row
          cur = conn.cursor()
          cur.execute(''' SELECT id_claim FROM claims WHERE name = ? AND date = ? ;''', [name, time])
          r = cur.fetchone()
          conn.commit()
          id_claim = "{:05d}".format(r["id_claim"])
          return render_template("claimed.html",id_claim = r["id_claim"], name = name)
        else:
          return render_template("index.html")


@app.route("/claims", methods=["GET", "POST"])
@login_required
def claims():
    conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
    m_1 = folium.Map(location=(lat, long), zoom_start=12)
    cursor = conn.execute(''' SELECT latitude, longitude, reason, id_claim, address FROM claims;''')
    date_ = []
    for row in cursor:
        if row[2] == "Sanitary sewer overflow":
            date_.append([row[3], row[4]])
            folium.Marker(location=[row[0], row[1]], popup=row[2],
                          icon=folium.Icon(color='blue', prefix='fa')).add_to(m_1)
        else:
            date_.append([row[3], row[4]])
            folium.Marker(location=[row[0], row[1]], popup=row[2],
                          icon=folium.Icon(color='red', prefix='fa')).add_to(m_1)        
    conn.close()
    m_1.save('/content/drive/MyDrive/final/template/map.html')

    return render_template('claims.html',date_=date_)





@app.route("/register", methods=["GET", "POST"])
def register():
    """Register user"""
    if request.method == "POST":
        password = request.form.get("password")
        phash = generate_password_hash(password)
        code = request.form.get("code")
        # This verify if the code entered is the code stablished
        if code != "1234":
            return render_template("register.html", wrongcode="Enter a valid code")
        else:
            conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
            conn.execute("INSERT INTO users (username, hash) VALUES(?, ?)", [request.form.get("name"), phash])
            rows = conn.execute("""SELECT id FROM users WHERE username = ?""", [request.form.get("name")])
            row = rows.fetchone()
            session["user_id"] = row[0]
            conn.commit()
        # Redirect user to home page
            return redirect("/claims")
    else:
        return render_template("register.html")

@app.route("/login", methods=["GET", "POST"])
def login():
    """Log user in"""

    # Forget any user_id
    session.clear()

    # User reached route via POST (as by submitting a form via POST)
    if request.method == "POST":
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        # Query database for username
        conn.row_factory = sqlite3.Row
        cur = conn.cursor()
        msj = "Wrong user name or password"
        cur.execute("""SELECT * FROM users WHERE username = ?""", [request.form.get("name")])
        r = cur.fetchone()
        conn.commit()
        # Ensure username exists and password is correct
        if r == None:
            return redirect("/claims")
        if r["username"] == request.form.get("name") and check_password_hash(r["hash"], request.form.get("password")):
            # Remember which user has logged in
            session["user_id"] = r["id"]
            # Redirect user to home page
            return redirect("/claims")
        else:
            return render_template("login.html")

    # User reached route via GET (as by clicking a link or via redirect)
    else:
        return render_template("login.html")


@app.route('/CSV')
def CSV():
    f = open('/content/drive/MyDrive/final/outputs/claimsCSV.csv', 'w')
    conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
    cursor = conn.execute(''' SELECT  id_claim, reason, name, address, date FROM claims;''')
    # This generates a csv file
    writer = csv.writer(f)
    header = ['id_claim', 'reason', 'name', 'address', "date"]
    writer.writerow(header)
    # Then add each row in the csv file
    for row in cursor:
        writer.writerow(row)
    f.close()
    conn.close()
    # The csv is returned to the page to download it
    return send_file('/content/drive/MyDrive/final/outputs/claimsCSV.csv',
                     mimetype='application/x-csv',
                     attachment_filename='claimsCSV.csv',
                     as_attachment=True)


@app.route("/del", methods=["GET", "POST"])
@login_required
def delete():

    conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
    id_claim = request.form.get("data")
    id_claim = int(id_claim)
    conn.execute("DELETE from claims WHERE id_claim = ?", [id_claim])
    conn.commit()
    conn.close()
    claims()
    return render_template("del.html", id_claim=id_claim)


@app.route("/logout")
def logout():
    """Log user out"""

    # Forget any user_id
    session.clear()

    # Redirect user to login form
    return redirect("/")

@app.route('/map')
def map():
    return render_template('map.html')


@app.route("/ajax_index", methods=["GET", "POST"])
def ajax_index():
    print("holaaa")
    if request.method == "POST":
      # This seeks the asked direction
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        json_data = json.loads(request.data)
        gps1 = loc(json_data["direction"])
        gps1[0] = float(gps1[0])
        gps1[1] = float(gps1[1])
        m_1 = folium.Map(location=(gps1[0], gps1[1]), zoom_start=120)
        folium.Marker(location=[gps1[0], gps1[1]]).add_to(m_1)
        m_1.save('/content/drive/MyDrive/final/template/map.html')
        return render_template("get_.html")

 

@app.route("/Ajax", methods=["GET", "POST"])
def Ajax():
    print("hola")
    if request.method == "POST":
        conn = sqlite3.connect('/content/drive/MyDrive/final/claims.db', check_same_thread=False)
        reason_s = request.form.get("data")
        m_1 = folium.Map(location=(lat, long), zoom_start=12)
        cursor = conn.execute(''' SELECT latitude, longitude, reason, id_claim, address FROM claims;''')
        if len(reason_s) < 3:
            # If reason_s is a claim number,
            # it will be seeks for a the same claim id
            # and add a marker in the map and render it
            for row in cursor:
                n = int(reason_s)
                if n == row[3]:
                    m_1 = folium.Map(location=([row[0], row[1]]), zoom_start=120)
                    folium.Marker(location=[row[0], row[1]]).add_to(m_1)
                    m_1.save('/content/drive/MyDrive/final/template/map.html')
                    return render_template("get_.html")
            # If reason_s is a reason of claim, every claim will be marked in the map and then return the map
        else:
            for row in cursor:
              #
                if reason_s == row[2]:
                    folium.Marker(location=[row[0], row[1]], popup=row[2]).add_to(m_1)
                if reason_s == "all":
                    if row[2] == "Sanitary sewer overflow":
                        folium.Marker(location=[row[0], row[1]], popup=row[2],
                                      icon=folium.Icon(color='blue', prefix='fa')).add_to(m_1)
                    else:
                        folium.Marker(location=[row[0], row[1]], popup=row[2],
                                      icon=folium.Icon(color='red', prefix='fa')).add_to(m_1)
        conn.close()
        m_1.save('/content/drive/MyDrive/final/template/map.html')
        return render_template("get_.html")

def loc(direction):
    # https://stackoverflow.com/questions/52638926/geo-coding-for-intersection-street-using-python from here i have taken the idea for searching latitude and longitude from googlemaps
    try:
        # cookies={'CONSENT': 'YES+cb.20210328-17-p0.en-GB+FX+{}'.format(random.randint(100, 999)) is used to accept cookies
        # taken from here https://stackoverflow.com/a/66940841/16053204
        res = requests.get('https://www.google.com/maps/place/'+direction+",+Ciudad+de+Córdoba,+Argentina",
                           cookies={'CONSENT': 'YES+cb.20210328-17-p0.en-GB+FX+{}'.format(random.randint(100, 999))})
        return re.findall(r'll=(.*?)" item', res.text)[0].split(',')
    except IndexError:
        res = requests.get('https://www.google.com/maps/place/'+direction+",+Ciudad+de+Córdoba,+Argentina",
                           cookies={'CONSENT': 'YES+cb.20210328-17-p0.en-GB+FX+{}'.format(random.randint(100, 999))})
        rest = re.findall(r'null,null,-(.*?)]', res.text)[0].split(',')
        rest[0] = float(rest[0]) * -1
        return rest

app.run()



 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


INFO:werkzeug: * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://89b9-34-139-188-188.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:01] "GET /map HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:01] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:04] "GET /login HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:08] "POST /login HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:09] "GET /claims HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:10] "GET /map HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:13] "GET /CSV HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:37] "GET /logout HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:38] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:41:38] "GET /map HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:42:00] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:42:01] "GET /map HTTP/

holaaa


...

INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:13] "POST /ajax_index HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:14] "GET /map HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:18] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:21] "GET /register HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:39] "POST /register HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:39] "GET /claims HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:39] "GET /map HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:47] "POST /Ajax HTTP/1.1" 200 -
ERROR:__main__:Exception on /ajax_index [POST]
Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "/usr/local/lib/python3.8/dist-packages/flask/app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/usr/l

hola
holaaa


INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:47] "GET /map HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:51] "POST /Ajax HTTP/1.1" 200 -


hola


INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:51] "GET /map HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:54] "POST /del HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:56:57] "GET /CSV HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:57:03] "GET /logout HTTP/1.1" 302 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:57:04] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [10/Feb/2023 21:57:04] "GET /map HTTP/1.1" 200 -
